# PROJECT
## Applications of Big Data

### 0. Importing libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

### 3. Data Modeling

#### 3.1. Dự đoán xu hướng tuyển dụng

In [2]:
df = pd.read_csv('dataset/data_clean.csv')
df.head()

,Job Id,Qualifications,location,Country,Work Type,Company Size,Job Posting Date,Preference,Job Title,Role,...,Job Description,Benefits,skills,Responsibilities,Company,Salary_avg,Experience_avg,Day,Month,Year
0,1089843540111562,M.Tech,Douglas,Isle of Man,Intern,26801,2022-04-24,Female,Digital Marketing Specialist,Social Media Manager,...,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,79.0,10.0,24,4,2022
1,398454096642776,BCA,Ashgabat,Turkmenistan,Intern,100340,2022-12-19,Female,Web Developer,Frontend Web Developer,...,Frontend Web Developers design and implement u...,"{'Health Insurance, Retirement Plans, Paid Tim...","HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",PNC Financial Services Group,86.0,7.0,19,12,2022
2,481640072963533,PhD,Macao,"Macao SAR, China",Temporary,84525,2022-09-14,Male,Operations Manager,Quality Control Manager,...,Quality Control Managers establish and enforce...,"{'Legal Assistance, Bonuses and Incentive Prog...",Quality control processes and methodologies St...,Establish and enforce quality control standard...,United Services Automobile Assn.,82.5,6.0,14,9,2022
3,688192671473044,PhD,Porto-Novo,Benin,Full-Time,129896,2023-02-25,Female,Network Engineer,Wireless Network Engineer,...,"Wireless Network Engineers design, implement, ...","{'Transportation Benefits, Professional Develo...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Hess,78.0,7.5,25,2,2023
4,117057806156508,MBA,Santiago,Chile,Intern,53944,2022-10-11,Female,Event Manager,Conference Manager,...,A Conference Manager coordinates and manages c...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,Cairn Energy,75.5,6.5,11,10,2022


##### 3.1.1. Data Preparing

In [3]:
df['Job Posting Date'] = pd.to_datetime(df['Job Posting Date'])

In [4]:
df['Job Posting Date'].describe()

C:\Users\HoaTX\AppData\Local\Temp\ipykernel_2992\2007897400.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df['Job Posting Date'].describe()


count                 1615940
unique                    731
top       2021-11-14 00:00:00
freq                     2378
first     2021-09-15 00:00:00
last      2023-09-15 00:00:00
Name: Job Posting Date, dtype: object

Thêm cột `Target1` để dự đoán xu hướng trong thị trường tuyển dụng dựa trên dữ liệu.

Ngày đăng tuyển dụng cuối cùng được thu thập là `15/09/2023` ta sẽ chọn giá trị cho cột `Target1` là `1` với những công việc tuyển dụng từ ngày `02/08/2023` và ngược lại là `0`. Nghĩa là trong `45` ngày gần nhất đăng tuyển để dự đoán xu hướng tuyển dụng.

In [5]:
df['Target1'] = df['Job Posting Date'].apply(lambda x: 1 if x >= pd.to_datetime('2023-08-02') else 0)

In [6]:
features = ['Work Type', 'Company Size', 'Job Title', 'Experience_avg']
df_model = df[features + ['Target1']]

In [7]:
df_model = pd.get_dummies(df_model, columns=['Job Title', 'Work Type'], prefix=['Job_Title', 'Work_Type'], drop_first=True)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df_model.drop('Target1', axis=1), df_model['Target1'], test_size=0.2, random_state=42)

##### 3.1.2. Naive Bayes

Mô hình `MultinomialNB` là một mô hình học máy có giám sát sử dụng cho bài toán phân loại văn bản. Nó dựa trên nguyên tắc của `Naive Bayes` và được sử dụng cho các tập dữ liệu có các đặc trưng rời rạc, chẳng hạn như đếm số lần xuất hiện của các từ trong văn bản. Mô hình xây dựng bộ "từ vựng" từ dữ liệu huấn luyện và tính xác suất xuất hiện của các từ trong "từ vựng" trong các lớp khác nhau. Sau đó, sử dụng các xác suất này để huấn luyện mô hình và phân loại các văn bản mới bằng cách dự đoán lớp có xác suất cao nhất cho mỗi văn bản. 

In [9]:
model1 = MultinomialNB()
model1.fit(X_train, y_train)
y_pred1 = model1.predict(X_test)

acc_1 = accuracy_score(y_test, y_pred1)
prec_1 = precision_score(y_test, y_pred1)
rec_1 = recall_score(y_test, y_pred1)
f1_1 = f1_score(y_test, y_pred1)

print('accuracy_score: ', acc_1)
print('precision_score: ', prec_1)
print('recall_score: ', rec_1)
print('f1_score: ', f1_1)

e:\Conda\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy_score:  0.9381103258784361
precision_score:  0.0
recall_score:  0.0
f1_score:  0.0


##### 3.1.3. Decision Tree Classifier

`Decision Tree Classifier` là một thuật toán học máy được sử dụng trong phân loại. Nó hoạt động bằng cách xây dựng một cây quyết định, tại mỗi nút của cây, thuật toán chọn thuộc tính tốt nhất để phân chia dữ liệu thành các tập con dựa trên câu hỏi. Quá trình này lặp lại cho đến khi đáp ứng điều kiện dừng. Khi cây đã xây dựng, nó có thể dự đoán lớp của dữ liệu mới bằng cách đi từ gốc cây đến nút lá tương ứng. 

In [ ]:
model2 = DecisionTreeClassifier()
model2.fit(X_train, y_train)
y_pred2 = model2.predict(X_test)

acc_2 = accuracy_score(y_test, y_pred2)
prec_2 = precision_score(y_test, y_pred2)
rec_2 = recall_score(y_test, y_pred2)
f1_2 = f1_score(y_test, y_pred2)

print('accuracy_score: ', acc_2)
print('precision_score: ', prec_2)
print('recall_score: ', rec_2)
print('f1_score: ', f1_2)

accuracy_score:  0.8802678317264255
precision_score:  0.06228341294371078
recall_score:  0.0664933506649335
f1_score:  0.06431956668923494


##### 3.1.4. Random Forest Classifier


Mô hình `RandomForestClassifier` là một mô hình học máy sử dụng trong bài toán phân loại. Nó dựa trên học kết hợp và sử dụng nhiều cây quyết định để thực hiện phân loại. Quá trình hoạt động bao gồm xây dựng nhiều cây quyết định độc lập, sau đó kết hợp kết quả dự đoán của chúng để đưa ra kết quả phân loại cuối cùng. `RandomForestClassifier` giúp tránh `overfitting` và tổng quát hóa tốt hơn do sử dụng nhiều cây và kết hợp kết quả của chúng. Các tham số quan trọng cần được điều chỉnh để cải thiện hiệu suất của mô hình. Mô hình này thường được sử dụng trong các bài toán phân loại phức tạp và đa dạng hoặc khi cần cải thiện tính tổng quát của mô hình.

In [ ]:
model3 = RandomForestClassifier(max_depth=10)
model3.fit(X_train, y_train)
y_pred3 = model3.predict(X_test)

acc_3 = accuracy_score(y_test, y_pred3)
prec_3 = precision_score(y_test, y_pred3)
rec_3 = recall_score(y_test, y_pred3)
f1_3 = f1_score(y_test, y_pred3)

print('accuracy_score: ', acc_3)
print('precision_score: ', prec_3)
print('recall_score: ', rec_3)
print('f1_score: ', f1_3)

e:\Conda\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy_score:  0.9381103258784361
precision_score:  0.0
recall_score:  0.0
f1_score:  0.0


In [ ]:
'''
estimators = range(100,1001,50)
best_acc = 0
best_k = 0
score=[]
models = []

for i in estimators:
    model_ = RandomForestClassifier(n_estimators=i)
    model_.fit(X_train, y_train)

    acc_score_rfc = accuracy_score(y_test, model_.predict(X_test))
    score.append(acc_score_rfc)
    
    models.append(model_)

    if acc_score_rfc  > best_acc:
        best_acc = acc_score_rfc
        best_k = i

print(f"best C: {best_k}, best accuracy: {best_acc}")
'''

'\nestimators = range(100,1001,50)\nbest_acc = 0\nbest_k = 0\nscore=[]\nmodels = []\n\nfor i in estimators:\n    model_ = RandomForestClassifier(n_estimators=i)\n    model_.fit(X_train, y_train)\n\n    acc_score_rfc = acc_score(y_test, model_.predict(X_test))\n    score.append(acc_score_rfc)\n    \n    models.append(model_)\n\n    if acc_score_rfc  > best_acc:\n        best_acc = acc_score_rfc\n        best_k = i\n\nprint(f"best C: {best_k}, best accuracy: {best_acc}")\n'

In [ ]:
#print(f"best C: {best_k}, best accuracy: {best_acc}")

In [ ]:
#plt.plot([*estimators],score, marker='o')
#plt.title('Precision Score')

##### 3.1.5. Logistic Regression   

Mô hình `LogisticRegression` là một mô hình học máy sử dụng cho bài toán phân loại hai lớp (binary classification). Nó tính toán xác suất của điểm dữ liệu thuộc vào lớp positive sử dụng hàm sigmoid. Trong quá trình huấn luyện, mô hình điều chỉnh các trọng số của đặc trưng để tối ưu hóa việc dự đoán xác suất cho các điểm dữ liệu huấn luyện bằng cách tối thiểu hóa hàm mất mát (loss function), như hàm `cross-entropy`. Sau đó, mô hình có thể dự đoán lớp của các điểm dữ liệu mới dựa trên xác suất và ngưỡng quyết định. Mô hình `LogisticRegression` là một lựa chọn phổ biến và hiệu quả cho các bài toán phân loại hai lớp.

In [ ]:
model4 = LogisticRegression()
model4.fit(X_train, y_train)
y_pred4 = model4.predict(X_test)

acc_4 = accuracy_score(y_test, y_pred4)
prec_4 = precision_score(y_test, y_pred4)
rec_4 = recall_score(y_test, y_pred4)
f1_4 = f1_score(y_test, y_pred4)

print('accuracy_score: ', acc_4)
print('precision_score: ', prec_4)
print('recall_score: ', rec_4)
print('f1_score: ', f1_4)

e:\Conda\Anaconda\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy_score:  0.9381103258784361
precision_score:  0.0
recall_score:  0.0
f1_score:  0.0


##### 3.1.6. Gradient Boosting Classifier

`Gradient Boosting` là một phương pháp học máy, tập trung vào xây dựng một chuỗi các mô hình dự đoán liên tiếp để cải thiện sai số. Quá trình hoạt động bao gồm xây dựng một mô hình ban đầu, tính sai số, tạo mô hình mới để dự đoán sai số, cộng hưởng dự đoán và lặp lại quá trình cho đến khi đạt được sai số mong muốn hoặc số lượng mô hình cần thiết. `Gradient Boosting` tối ưu hóa hàm mất mát thông qua `Gradient Descent` để điều chỉnh tham số của các mô hình. Các thư viện phổ biến để thực hiện `Gradient Boosting` bao gồm XGBoost, LightGBM và `scikit-learn`.

In [ ]:
model5 = GradientBoostingClassifier()
model5.fit(X_train, y_train)
y_pred5 = model5.predict(X_test)

acc_5 = accuracy_score(y_test, y_pred5)
prec_5 = precision_score(y_test, y_pred5)
rec_5 = recall_score(y_test, y_pred5)
f1_5 = f1_score(y_test, y_pred5)

print('accuracy_score: ', acc_5)
print('precision_score: ', prec_5)
print('recall_score: ', rec_5)
print('f1_score: ', f1_5)

accuracy_score:  0.9381010433555701
precision_score:  0.0
recall_score:  0.0
f1_score:  0.0


##### 3.1.7. Support Vector Machine

Mô hình `SVC` là một mô hình học máy được sử dụng trong bài toán phân loại. Nó dựa trên ý tưởng của máy vector hỗ trợ (Support Vector Machines - SVM) và tìm một đường ranh giới trong không gian đặc trưng để phân tách các lớp dữ liệu. `SVC` sử dụng các điểm dữ liệu gần nhất với đường ranh giới (hyperplane) để tối đa hóa khoảng cách giữa các lớp. Tham số `C` (mức độ chấp nhận lỗi) điều chỉnh mức độ "mềm" của đường ranh giới. Nó được sử dụng phổ biến trong các bài toán phân loại phức tạp, nhất là khi dữ liệu không phân tách tuyến tính. Tuy nhiên, việc huấn luyện `SVC` có thể tốn nhiều thời gian đối với các tập dữ liệu lớn.

In [ ]:
'''
C = [0.01,0.1, 0.5, 1, 1.5, 2, 5, 10]
#k = ['linear', 'poly', 'rbf', 'sigmoid']
best_score = 0
best_C = ''
score_1 = []
#best_k = ''
#score_2 = []
for i in C:
    model6 = SVC(C=i)
    model6.fit(X_train, y_train)

    accuracy_score_svc = accuracy_score(y_test, model6.predict(X_test))
    score_1.append(accuracy_score_svc)
    #score_2.append(accuracy_score_svc)

    if accuracy_score_svc  > best_score:
        best_score = accuracy_score_svc 
        best_C = i
        #best_k = i

    print(f"C: {i}, accurary: {accuracy_score_svc }")
    #print(f"k: {i}, accurary: {accuracy_score_svc }")

print(f"best C: {best_C}, best accurary: {best_score}")
#print(f"best k: {best_k}, best accurary: {best_score}")
'''

In [ ]:
#plt.plot(C,score_1);

In [ ]:
model6 = SVC(C=1)
model6.fit(X_train, y_train)
y_pred = model6.predict(X_test)

acc_6 = accuracy_score(y_test, y_pred)
pre_6 = precision_score(y_test, y_pred)
rec_6 = recall_score(y_test, y_pred)
f1_6 = f1_score(y_test, y_pred)

print("Accuracy: ", acc_6)
print("Precision: ", pre_6)
print("Recall: ", rec_6)
print("F1: ", f1_6)